In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import hamming_loss, f1_score, jaccard_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim


In [2]:

def sinusoidal_encoding(coordinates):
    if coordinates.shape[1] != 2:
        raise ValueError("Input array must have two columns (lon, lat)")

    lon = 2 * (coordinates[:, 0] - coordinates[:, 0].min()) / (coordinates[:, 0].max() - coordinates[:, 0].min()) - 1
    lat = 2 * (coordinates[:, 1] - coordinates[:, 1].min()) / (coordinates[:, 1].max() - coordinates[:, 1].min()) - 1


    encoded_lon = np.column_stack([np.sin(np.pi * lon), np.cos(np.pi * lon)])
    encoded_lat = np.column_stack([np.sin(np.pi * lat), np.cos(np.pi * lat)])

    encoded_vector = np.concatenate([encoded_lon, encoded_lat], axis=1)

    return encoded_vector


In [5]:
# loading training data    
data = np.load('data/species_train.npz')
train_locs = data['train_locs']  # 2D array, rows are number of datapoints and 
                                 # columns are "latitude" and "longitude"
train_ids = data['train_ids']    # 1D array, entries are the ID of the species 
                                 # that is present at the corresponding location in train_locs
species = data['taxon_ids']      # list of species IDe. Note these do not necessarily start at 0 (or 1)
species_names = dict(zip(data['taxon_ids'], data['taxon_names']))  # latin names of species 

# loading test data 
data_test = np.load('data/species_test.npz', allow_pickle=True)
test_locs = data_test['test_locs']    # 2D array, rows are number of datapoints 
                                      # and columns are "latitude" and "longitude"
# data_test['test_pos_inds'] is a list of lists, where each list corresponds to 
# the indices in test_locs where a given species is present, it can be assumed 
# that they are not present in the other locations 
test_pos_inds = dict(zip(data_test['taxon_ids'], data_test['test_pos_inds']))    

In [6]:
encoder = OneHotEncoder(sparse_output=False)

# Fittinh and transforming the train_ids to one-hot encoded labels
train_ids_reshaped = train_ids.reshape(-1, 1)
one_hot_encoded_ids = encoder.fit_transform(train_ids_reshaped)
X = sinusoidal_encoding(train_locs)
y = one_hot_encoded_ids
X.shape, y.shape

NameError: name 'OneHotEncoder' is not defined

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=41)
X_train = torch.FloatTensor(X_train)
X_val = torch.FloatTensor(X_val)
y_train = torch.FloatTensor(y_train)
y_val = torch.FloatTensor(y_val)
print("X_train size : ", X_train.data.size())
print("y_train size : ", y_train.data.size())
print("X_val size : ", X_val.data.size())
print("y_val size : ", y_val.data.size())

X_train size :  torch.Size([217629, 4])
y_train size :  torch.Size([217629, 500])
X_val size :  torch.Size([54408, 4])
y_val size :  torch.Size([54408, 500])


Model with standard BCE LOSS for Multi Label Classification

In [7]:
class MultiLabelNeuralNet_BCE(nn.Module):
    def __init__(self, in_features=4, num_classes=500):
        super().__init__()
        self.layer_one = nn.Linear(in_features, 32)
        self.layer_two = nn.Linear(32,64)
        self.layer_three = nn.Linear(64,128)
        self.layer_four = nn.Linear(128,200)
        self.output_layer = nn.Linear(200, num_classes)

        nn.init.xavier_uniform_(self.layer_one.weight)
        nn.init.xavier_uniform_(self.layer_two.weight)
        nn.init.xavier_uniform_(self.layer_three.weight)
        nn.init.xavier_uniform_(self.layer_four.weight)
        nn.init.xavier_uniform_(self.output_layer.weight)
        

    def forward(self, x):
        x = nn.functional.relu(self.layer_one(x))
        x = nn.functional.relu(self.layer_two(x))
        x = nn.functional.relu(self.layer_three(x))
        x = nn.functional.relu(self.layer_four(x))
        x = self.output_layer(x)
        return x

model = MultiLabelNeuralNet_BCE()  

# Define the BCEWithLogitsLoss loss
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.007)

num_epochs = 2
train_losses = []
validation_losses =[]
for epoch in range(num_epochs):
    model.train()
    # Forward pass
    outputs = model.forward(X_train)

    # Calculate the loss
    loss = criterion(outputs,y_train)

    train_losses.append(loss.detach().numpy())

    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Update the weights
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model.forward(X_val)
        val_loss = criterion(val_outputs, y_val)
        validation_losses.append(val_loss.item())

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Train Loss: {loss.item()}, Val Loss: {val_loss.item()}')


Epoch: 0, Train Loss: 0.6933140158653259, Val Loss: 0.6440598368644714
Epoch: 1, Train Loss: 0.6440410017967224, Val Loss: 0.5433555245399475


In [8]:
def create_one_hot_encoded_matrix(test_locs, test_pos_inds):

    unique_species_ids = sorted(test_pos_inds.keys())

    one_hot_encoded_matrix = np.zeros((len(test_locs), len(unique_species_ids)))

    species_id_to_index = {species_id: index for index, species_id in enumerate(unique_species_ids)}

    for species_id, indices in test_pos_inds.items():
        one_hot_encoded_matrix[indices, species_id_to_index[species_id]] = 1

    return one_hot_encoded_matrix

one_hot_test_data = create_one_hot_encoded_matrix(test_locs, test_pos_inds)

In [12]:
X_t = sinusoidal_encoding(test_locs)
y_t = one_hot_test_data
X.shape, y.shape
X_tt = torch.FloatTensor(X_t)
y_tt = torch.FloatTensor(y_t)
print("X_test size : ", X_tt.data.size())
print("y_test size : ", y_tt.data.size())


X_test size :  torch.Size([288122, 4])
y_test size :  torch.Size([288122, 500])


In [4]:
with torch.no_grad():
    y_test = model.forward(X_tt)
    test_loss = criterion(y_test,y_tt)

test_loss

NameError: name 'torch' is not defined

In [3]:
threshold = 0.5
predictions = (torch.sigmoid(y_test) > threshold).float()

predictions_np = predictions.numpy()
y_true_np = y_tt.numpy()

predictions = predictions.numpy()  
true_labels= y_tt.numpy() 

y_true = true_labels
y_pred = predictions

NameError: name 'torch' is not defined

In [20]:
import warnings
# Hamming Loss
hamming_loss = metrics.hamming_loss(y_true, y_pred)
print(f'Hamming Loss: {hamming_loss:.4f}')
hamming_accuracy = 1 - hamming_loss
print(f'Hamming Accuracy: {hamming_accuracy:.4f}')

# Jaccard Score
jaccard_score_micro= metrics.jaccard_score(y_true, y_pred, average='micro')
print(f'Jaccard Score_micro: {jaccard_score_micro:.4f}')
jaccard_score_macro= metrics.jaccard_score(y_true, y_pred, average='macro')
print(f'Jaccard Score-macro: {jaccard_score_macro:.4f}')

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(classification_report(y_true,y_pred, target_names = species.sort()))

Hamming Loss: 0.0195
Hamming Accuracy: 0.9805
Jaccard Score_micro: 0.0035
Jaccard Score-macro: 0.0004


c:\Users\irfan_lkd13dt\miniconda3\envs\aml\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       619
           1       0.00      0.00      0.00      2234
           2       0.00      0.00      0.00      2926
           3       0.00      0.00      0.00       685
           4       0.00      0.00      0.00       850
           5       0.00      0.00      0.00       631
           6       0.00      0.00      0.00      4960
           7       0.00      0.00      0.00      6939
           8       0.00      0.00      0.00       253
           9       0.00      0.00      0.00      3012
          10       0.00      0.00      0.00       814
          11       0.00      0.00      0.00      4060
          12       0.00      0.00      0.00      1759
          13       0.00      0.00      0.00      1605
          14       0.00      0.00      0.00       277
          15       0.00      0.00      0.00      4819
          16       0.00      0.00      0.00      3269
          17       0.00    

c:\Users\irfan_lkd13dt\miniconda3\envs\aml\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(predictions.shape[1]):
    fpr[i], tpr[i], _ = roc_curve(true_labels[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


all_fpr = np.unique(np.concatenate([fpr[i] for i in range(predictions.shape[1])]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(predictions.shape[1]):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= predictions.shape[1]

roc_auc["macro"] = auc(all_fpr, mean_tpr)


plt.figure(figsize=(10, 6))
plt.plot(all_fpr, mean_tpr, label=f'Macro-average ROC Curve (AUC = {roc_auc["macro"]:.2f})', linestyle='--', linewidth=2)
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random', alpha=0.8)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Macro-average ROC Curve')
plt.legend(loc='lower right')
plt.show()


NameError: name 'predictions' is not defined